In [19]:
import logging
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import Word2Vec
from numpy import isin

SEED = 42
np.random.seed(SEED)

news_marlesson = pd.read_csv('datasets/kaggle_marlesson_noticias_folha_sp.csv')

In [20]:
news_marlesson.drop(columns=['subcategory'], inplace=True)

In [21]:
news_marlesson.head()

,title,text,date,category,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [22]:
print('sem drop de NaN: ', news_marlesson.shape)
news_marlesson.dropna(subset=['text'],inplace=True)
print('com drop de NaN: ',news_marlesson.shape )


sem drop de NaN:  (167053, 5)
com drop de NaN:  (166288, 5)


In [23]:
news_marlesson.category.value_counts()

poder                           22022
colunas                         21619
mercado                         20970
esporte                         19730
mundo                           17130
cotidiano                       16967
ilustrada                       15617
opiniao                          4525
paineldoleitor                   4011
saopaulo                         3955
tec                              2260
tv                               2123
educacao                         2118
turismo                          1903
ilustrissima                     1409
ciencia                          1335
equilibrioesaude                 1312
sobretudo                        1057
bbc                               980
folhinha                          875
empreendedorsocial                841
comida                            828
asmais                            547
ambiente                          491
seminariosfolha                   379
serafina                          331
o-melhor-de-

In [24]:
s = news_marlesson['category'].value_counts()
news_marlesson = news_marlesson[news_marlesson.isin(s.index[s >= 1000]).values]

In [25]:
news_marlesson.head()

,title,text,date,category,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,http://www1.folha.uol.com.br/mercado/2017/10/1...


$ si = [fmax + (ti - tmin)] $

In [26]:
def clean_text(doc):
    result = ""

    for token in doc:
        is_valid = not token.is_stop and token.is_alpha

        if is_valid:
            result += f"{token} "

    return result.rstrip()

In [27]:
news_marlesson['tamanho_noticia'] = news_marlesson.text.str.len()

In [28]:
print(news_marlesson['tamanho_noticia'].mean(), news_marlesson['tamanho_noticia'].max());

2718.365037516478 61154


In [29]:
news_marlesson.sort_values(by=['category', 'tamanho_noticia'], ascending=True, inplace=True);

In [30]:
news_splited = np.array_split(news_marlesson, 100)

In [31]:
for news in news_splited:
    news_to_clean = (news.lower() for news in news["text"])

In [32]:
nlp = spacy.load("pt_core_news_sm")

In [33]:
news_cleaned = [clean_text(doc) for doc in nlp.pipe(news_to_clean,  batch_size = 10000, n_process = -1)]

In [34]:
news_to_list = pd.DataFrame({'news': news_cleaned})
news_to_list.drop_duplicates(inplace=True)

In [35]:
news_list = [news.split(' ') for news in news_to_list.news]

In [36]:
logging.basicConfig(format='%(asctime)s : - %(message)s', level=logging.INFO)

word_2_vec_skipgram_model = Word2Vec(sg = 1, window = 100, min_count = 5, vector_size = 300, alpha = 0.03, min_alpha=0.007)
word_2_vec_skipgram_model.build_vocab(news_list, progress_per=5000)
word_2_vec_skipgram_model.train(news_list, total_examples = word_2_vec_skipgram_model.corpus_count, epochs = 1000)

2022-11-10 16:52:27,653 : - Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2022-11-10T16:52:27.653698', 'gensim': '4.2.0', 'python': '3.7.13 (default, Oct 18 2022, 18:57:03) \n[GCC 11.2.0]', 'platform': 'Linux-5.15.68.1-microsoft-standard-WSL2-x86_64-with-debian-bullseye-sid', 'event': 'created'}
2022-11-10 16:52:27,655 : - collecting all words and their counts
2022-11-10 16:52:27,656 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-11-10 16:52:27,683 : - collected 20754 word types from a corpus of 107972 raw words and 1589 sentences
2022-11-10 16:52:27,685 : - Creating a fresh vocabulary
2022-11-10 16:52:27,707 : - Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 4486 unique words (21.62% of original 20754, drops 16268)', 'datetime': '2022-11-10T16:52:27.707510', 'gensim': '4.2.0', 'python': '3.7.13 (default, Oct 18 2022, 18:57:03) \n[GCC 11.2.0]', 'platform': 'Linux-5.15.68.1-microsoft-standa

(79483444, 107972000)

In [37]:
# word_2_vec_skipgram_model.wv.save_word2vec_format('models/marlesson/sg_300.txt', binary=False)

In [38]:
# logging.basicConfig(format='%(asctime)s : - %(message)s', level=logging.INFO)

# word_2_vec_skipgram_model = Word2Vec(sg = 1, window = 4, min_count = 5, vector_size = 600, alpha = 0.03, min_alpha=0.007)
# word_2_vec_skipgram_model.build_vocab(news_list, progress_per=5000)
# word_2_vec_skipgram_model.train(news_list, total_examples = word_2_vec_skipgram_model.corpus_count, epochs = 1000)
# word_2_vec_skipgram_model.wv.save_word2vec_format('models/marlesson/sg_600.txt', binary=False)

In [39]:
# logging.basicConfig(format='%(asctime)s : - %(message)s', level=logging.INFO)

# word_2_vec_skipgram_model = Word2Vec(sg = 1, window = 4, min_count = 5, vector_size = 1000, alpha = 0.03, min_alpha=0.007)
# word_2_vec_skipgram_model.build_vocab(news_list, progress_per=5000)
# word_2_vec_skipgram_model.train(news_list, total_examples = word_2_vec_skipgram_model.corpus_count, epochs = 1000)
# word_2_vec_skipgram_model.wv.save_word2vec_format('models/marlesson/sg_1000.txt', binary=False)